kmeans
dataset's batch 20
data's batch 50

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from psc_firework import PSC
import time
import torch
import random

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

df = pd.read_csv('firework.csv')
print(df.head(10))
action = {'allow': 1, 'deny': 2, 'drop': 3, 'reset-both': 4}
df['Action'] = df['Action'].map(action)
y = df['Action'].values
print(y)
x_data = df.drop(['Action'], axis = 1).values
# x_data = df.drop(['Action', 'Elapsed Time (sec)', 'pkts_sent', 'pkts_received'], axis = 1).values
print(x_data)
print("x_data type", type(x_data))
print("y type", type(y))
print("x_data shape", x_data.shape)
print("y shape", y.shape)

scaler = sklearn.preprocessing.StandardScaler().fit(x_data)
x = scaler.transform(x_data)
# x = (x_data - np.min(x_data)) / (np.max(x_data) - np.min(x_data))
print(x)
print("x shape ", x.shape)

class Net1(nn.Module):
    def __init__(self, out):
        super(Net1, self).__init__()
        self.output = out
        # Define the layers
        self.fc1 = nn.Linear(8, 32)
        self.fc2 = nn.Linear(32, 64)
        self.fc3 = nn.Linear(64, 32)
        self.output_layer = nn.Linear(32, self.output)

        self.relu = nn.ReLU()

    def forward(self, x):

        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fc3(x)
        x = self.output_layer(x)
        return x


In [ ]:
silhouette_avg_kmeans = []
calinski_harabasz_kmeans = []
homogeneity_kmeans = []
v_measure_kmeans = []
# Davies-Boulding
for i in range(2, 22, 2):
    print(f"n_clusters = {i}")
    model = Net1(out=i)
    kmeans = KMeans(n_clusters=4, init='random', n_init='auto', algorithm='elkan')
    time1 = round(time.time()*1000)
    x_fit_kmeans = kmeans.fit(x)
    x_predict_kmeans = kmeans.predict(x)
    time2 = round(time.time()*1000)
    print(f"PSC time spent: {time2 - time1} milliseconds")
    silhouette_avg_kmeans.append(metrics.silhouette_score(x, x_fit_kmeans.labels_))
    print(f"silhouette_avg: {silhouette_avg_kmeans[-1]}")
    calinski_harabasz_kmeans.append(metrics.calinski_harabasz_score(x, x_fit_kmeans.labels_))
    print(f"calinski_harabasz: {calinski_harabasz_kmeans[-1]}")
    homogeneity_kmeans.append(metrics.homogeneity_score(y, x_predict_kmeans))
    print(f"homogeneity: {homogeneity_kmeans[-1]}")
    v_measure_kmeans.append(metrics.v_measure_score(y, x_predict_kmeans))
    print(f"v_measure: {v_measure_kmeans[-1]}\n")

In [ ]:
print(silhouette_avg_kmeans)
plt.title("silhouette_avg_kmeans")
plt.xlabel("n_clusters")
xx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
silhouette_avg_kmeans.insert(0, 0)
plt.xticks(ticks=range(0, 11), labels=xx)
plt.plot(silhouette_avg_kmeans)

In [ ]:
print(calinski_harabasz_kmeans)
plt.title("calinski_harabasz_kmeans")
plt.xlabel("n_clusters")
xx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
calinski_harabasz_kmeans.insert(0, 0)
plt.xticks(ticks=range(0, 11), labels=xx)
plt.plot(calinski_harabasz_kmeans)

In [ ]:
print(homogeneity_kmeans)
plt.title("homogeneity_kmeans")
plt.xlabel("n_clusters")
xx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
homogeneity_kmeans.insert(0, 0)
plt.xticks(ticks=range(0, 11), labels=xx)
plt.plot(homogeneity_kmeans)

In [ ]:
print(v_measure_kmeans)
plt.title("v_measure_kmeans")
plt.xlabel("n_clusters")
xx = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20]
v_measure_kmeans.insert(0, 0)
plt.xticks(ticks=range(0, 11), labels=xx)
plt.plot(v_measure_kmeans)

In [ ]:
print('Intrinsic measures')
Silhouette_Score=metrics.silhouette_score(x, x_fit_kmeans.labels_, metric = 'euclidean')
print('Silhouette Score:',Silhouette_Score)

CH_score = metrics.calinski_harabasz_score(x, x_fit_kmeans.labels_)
print('Calinski Harabasz Score:',CH_score)
print('-----------------------------------------------')
print('Extrinsic measures')
H_Score=metrics.homogeneity_score(y, x_predict_kmeans)
print('homogeneity_score:',H_Score)

v_measure_score=metrics.v_measure_score(y, x_predict_kmeans)
print('v_measure_score:',v_measure_score)